In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.ensemble import VotingClassifier, StackingClassifier

import warnings
warnings.filterwarnings("ignore")

In [27]:
df = pd.read_csv('diabetes_merged_dataset.csv')
df

,gender,age,hypertension,bmi,blood_glucose_level,diabetes,HbA1c_level
0,Female,0.08,0,11.88,80,0,5.7
1,Female,0.08,0,12.22,130,0,3.5
2,Female,0.08,0,12.50,155,0,4.5
3,Female,0.08,0,12.74,140,0,3.5
4,Female,0.08,0,12.77,160,0,4.5
...,...,...,...,...,...,...,...
230129,Other,47.00,0,36.76,90,0,6.6
230130,Other,47.00,0,36.76,200,0,3.5
230131,Other,52.00,0,31.63,159,0,6.6
230132,Other,53.00,0,27.32,160,0,6.6


In [28]:
df.describe(include='all')

,gender,age,hypertension,bmi,blood_glucose_level,diabetes,HbA1c_level
count,230134,230134.000000,230134.000000,230134.000000,230134.000000,230134.000000,230134.000000
unique,3,NaN,NaN,NaN,NaN,NaN,NaN
top,Female,NaN,NaN,NaN,NaN,NaN,NaN
freq,135325,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,42.596647,0.039547,27.300237,135.832810,0.044283,5.463940
std,NaN,21.259874,0.194892,4.406273,37.915017,0.205723,1.027541
min,NaN,0.080000,0.000000,10.010000,80.000000,0.000000,3.500000
25%,NaN,26.000000,0.000000,27.320000,100.000000,0.000000,4.800000
50%,NaN,42.000000,0.000000,27.320000,140.000000,0.000000,5.800000
75%,NaN,58.000000,0.000000,27.320000,159.000000,0.000000,6.200000


In [29]:
df.isnull().sum()

gender                 0
age                    0
hypertension           0
bmi                    0
blood_glucose_level    0
diabetes               0
HbA1c_level            0
dtype: int64

In [30]:
X = df.drop(columns=['diabetes'])
y = df['diabetes']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [32]:
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

### Hyperparameter tuning and cross-validation for each model


In [33]:
# 1. RandomForestClassifier
rf_param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5]
}
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])
rf_grid = GridSearchCV(rf_pipeline, rf_param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1)
rf_grid.fit(X_train, y_train)
best_rf = rf_grid.best_estimator_

In [34]:
# evaluate RandomForestClassifier
rf_y_pred = best_rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_confusion = confusion_matrix(y_test, rf_y_pred)
rf_classification_report = classification_report(y_test, rf_y_pred)
print("RandomForestClassifier Accuracy:", rf_accuracy)
print("RandomForestClassifier Confusion Matrix:\n", rf_confusion)
print("RandomForestClassifier Classification Report:\n", rf_classification_report)

RandomForestClassifier Accuracy: 0.985660590522954
RandomForestClassifier Confusion Matrix:
 [[43930    59]
 [  601  1437]]
RandomForestClassifier Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99     43989
           1       0.96      0.71      0.81      2038

    accuracy                           0.99     46027
   macro avg       0.97      0.85      0.90     46027
weighted avg       0.99      0.99      0.98     46027



In [35]:
# 2. XGBClassifier
xgb_param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [3, 6],
    'classifier__learning_rate': [0.01, 0.1]
}
xgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
])
xgb_grid = GridSearchCV(xgb_pipeline, xgb_param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1)
xgb_grid.fit(X_train, y_train)
best_xgb = xgb_grid.best_estimator_

# evaluate XGBClassifier
xgb_y_pred = best_xgb.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
xgb_confusion = confusion_matrix(y_test, xgb_y_pred)
xgb_classification_report = classification_report(y_test, xgb_y_pred)
print("XGBClassifier Accuracy:", xgb_accuracy)
print("XGBClassifier Confusion Matrix:\n", xgb_confusion)
print("XGBClassifier Classification Report:\n", xgb_classification_report)

XGBClassifier Accuracy: 0.985334694853021
XGBClassifier Confusion Matrix:
 [[43964    25]
 [  650  1388]]
XGBClassifier Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99     43989
           1       0.98      0.68      0.80      2038

    accuracy                           0.99     46027
   macro avg       0.98      0.84      0.90     46027
weighted avg       0.99      0.99      0.98     46027



In [36]:
# 3. LGBMClassifier
lgbm_param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [-1, 10],
    'classifier__learning_rate': [0.01, 0.1]
}
lgbm_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LGBMClassifier(random_state=42))
])
lgbm_grid = GridSearchCV(lgbm_pipeline, lgbm_param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1)
lgbm_grid.fit(X_train, y_train)
best_lgbm = lgbm_grid.best_estimator_

# evaluate LGBMClassifier
lgbm_y_pred = best_lgbm.predict(X_test)
lgbm_accuracy = accuracy_score(y_test, lgbm_y_pred)
lgbm_confusion = confusion_matrix(y_test, lgbm_y_pred)
lgbm_classification_report = classification_report(y_test, lgbm_y_pred)
print("LGBMClassifier Accuracy:", lgbm_accuracy)
print("LGBMClassifier Confusion Matrix:\n", lgbm_confusion)
print("LGBMClassifier Classification Report:\n", lgbm_classification_report)

[LightGBM] [Info] Number of positive: 8153, number of negative: 175954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002522 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 403
[LightGBM] [Info] Number of data points in the train set: 184107, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.044284 -> initscore=-3.071837
[LightGBM] [Info] Start training from score -3.071837
LGBMClassifier Accuracy: 0.9853781476090121
LGBMClassifier Confusion Matrix:
 [[43989     0]
 [  673  1365]]
LGBMClassifier Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     43989
           1       1.00      0.67      0.80      2038

    accuracy                           0.99     46027
   macro avg       0.99      0.83      0.90     46027
weighted avg       0.99

In [ ]:
# 4. CatBoostClassifier
cat_param_grid = {
    'classifier__iterations': [100, 200],
    'classifier__depth': [3, 6],
    'classifier__learning_rate': [0.01, 0.1]
}
cat_pipeline = Pipeline([
    ('preprocessor', preprocessor),M
    ('classifier', CatBoostClassifier(verbose=0, random_state=42))
])
cat_grid = GridSearchCV(cat_pipeline, cat_param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1)
cat_grid.fit(X_train, y_train)
best_cat = cat_grid.best_estimator_

# Evaluate CatBoostClassifier
cat_y_pred = best_cat.predict(X_test)
cat_accuracy = accuracy_score(y_test, cat_y_pred)
cat_confusion = confusion_matrix(y_test, cat_y_pred)
cat_classification_report = classification_report(y_test, cat_y_pred)
print("CatBoostClassifier Accuracy:", cat_accuracy)
print("CatBoostClassifier Confusion Matrix:\n", cat_confusion)
print("CatBoostClassifier Classification Report:\n", cat_classification_report)

CatBoostClassifier Accuracy: 0.9853781476090121
CatBoostClassifier Confusion Matrix:
 [[43989     0]
 [  673  1365]]
CatBoostClassifier Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     43989
           1       1.00      0.67      0.80      2038

    accuracy                           0.99     46027
   macro avg       0.99      0.83      0.90     46027
weighted avg       0.99      0.99      0.98     46027



In [38]:
# Collect all best models for ensemble
best_models = [
    ('rf', best_rf),
    ('xgb', best_xgb),
    ('lgbm', best_lgbm),
    ('cat', best_cat)
]

In [39]:
# using votingclassifier
voting_clf = VotingClassifier(estimators=best_models, voting='soft')
voting_clf.fit(X_train, y_train)
# Evaluate the voting classifier
y_pred_voting = voting_clf.predict(X_test)
print("Voting Classifier Accuracy:", accuracy_score(y_test, y_pred_voting))
print("Voting Classifier Classification Report:\n", classification_report(y_test, y_pred_voting))

[LightGBM] [Info] Number of positive: 8153, number of negative: 175954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002685 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 403
[LightGBM] [Info] Number of data points in the train set: 184107, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.044284 -> initscore=-3.071837
[LightGBM] [Info] Start training from score -3.071837
Voting Classifier Accuracy: 0.9853781476090121
Voting Classifier Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     43989
           1       1.00      0.67      0.80      2038

    accuracy                           0.99     46027
   macro avg       0.99      0.83      0.90     46027
weighted avg       0.99      0.99      0.98     46027



In [40]:
# using stackingclassifier
stacking_clf = StackingClassifier(estimators=best_models, final_estimator=XGBClassifier(random_state=42, **xgb_grid.param_grid))
stacking_clf.fit(X_train, y_train)
# Evaluate the stacking classifier
y_pred_stacking = stacking_clf.predict(X_test)
print("Stacking Classifier Accuracy:", accuracy_score(y_test, y_pred_stacking))
print("Stacking Classifier Classification Report:\n", classification_report(y_test, y_pred_stacking))

[LightGBM] [Info] Number of positive: 8153, number of negative: 175954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002602 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 403
[LightGBM] [Info] Number of data points in the train set: 184107, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.044284 -> initscore=-3.071837
[LightGBM] [Info] Start training from score -3.071837
[LightGBM] [Info] Number of positive: 6522, number of negative: 140763
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004837 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 403
[LightGBM] [Info] Number of data points in the train set: 147285, number of used features: 7
[LightGBM] [Info] [b

In [41]:
final_model = stacking_clf

In [42]:
df

,gender,age,hypertension,bmi,blood_glucose_level,diabetes,HbA1c_level
0,Female,0.08,0,11.88,80,0,5.7
1,Female,0.08,0,12.22,130,0,3.5
2,Female,0.08,0,12.50,155,0,4.5
3,Female,0.08,0,12.74,140,0,3.5
4,Female,0.08,0,12.77,160,0,4.5
...,...,...,...,...,...,...,...
230129,Other,47.00,0,36.76,90,0,6.6
230130,Other,47.00,0,36.76,200,0,3.5
230131,Other,52.00,0,31.63,159,0,6.6
230132,Other,53.00,0,27.32,160,0,6.6


In [43]:
def calculate_BMI(bb, tb):
    """
    Calculate Body Mass Index (BMI) given body weight and height.
    
    Parameters:
    bb (float): Body weight in kilograms.
    tb (float): Height in meters.
    
    Returns:
    float: Calculated BMI.
    """
    return bb / (tb ** 2)

In [44]:
# making an input manual in terminal while running for prediction using final_model
def predict_diabetes(input_data):
    input_df = pd.DataFrame([input_data])
    # input_transformed = preprocessor.transform(input_df)
    # prediction = final_model.predict(input_transformed)
    prediction = final_model.predict(input_df)
    return "Diabetic" if prediction[0] == 1 else "Not Diabetic"

In [46]:
if __name__ == "__main__":
    gender = str(input("Enter gender: "))
    age = float(input("Enter age: "))
    body_weight = float(input("Enter body weight (kg): "))
    body_height = float(input("Enter body height (m): "))
    BMI = calculate_BMI(body_weight, body_height)
    hypertension = int(input("Enter hypertension (0 or 1): "))
    glucose_level = float(input("Enter glucose level: "))
    HbA1c_level = float(input("Enter HbA1c level: "))
    input_data = {
        'gender': gender,
        'age': age,
        'hypertension': hypertension,
        'bmi': BMI,
        'blood_glucose_level': glucose_level,
        'HbA1c_level': HbA1c_level
    }
    result = predict_diabetes(input_data)

In [47]:
result

'Not Diabetic'

In [48]:
# Save the final model
import joblib
joblib.dump(final_model, 'diabtes_predict_model.pkl')
print("Model saved as diabtes_predict_model.pkl")

Model saved as diabtes_predict_model.pkl
